In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import keras
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization
import keras.optimizers as optimizers
import matplotlib.pyplot as plt


Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [4]:
data_full = pd.read_csv('train.csv')

In [3]:
data_full.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [4]:
data_full['image_id']=data_full['image_id']+'.jpg'

In [5]:
data_full['image_id'][0]

'Train_0.jpg'

In [6]:
range(data_full.shape[0])

range(0, 1821)

In [7]:
train_data, val_data = train_test_split(data_full, test_size=0.2)

# Image processing

In [5]:
train_dir = '/Users/pawelpawlus/Documents/asseco/plant_pathology_classification/images/Train/'
image_size = tuple((300,300))
x = []
for i in tqdm(range(data_full.shape[0])):
    img = image.load_img(train_dir + data_full['image_id'][i]+'.jpg',color_mode='rgb',target_size=image_size)
    img = image.img_to_array(img)
    img = img/255
    x.append(img)
    

100%|██████████| 1821/1821 [00:37<00:00, 48.24it/s]


In [58]:
train_dir = '/Users/pawelpawlus/Documents/asseco/plant_pathology_classification/images/Train/'
image_size = 300
batch_size = 1

train_data_gen = image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.5,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    brightness_range=[0.5, 1])
    

In [59]:

train_generator = train_data_gen.flow_from_dataframe(
    train_data,
    train_dir,
    target_size=(image_size,image_size),
    x_col = 'image_id',
    y_col = ['healthy','multiple_diseases','rust','scab'],
    class_mode = 'raw',
    shuffle=False,
    subset='training',
    batch_size=batch_size)

Found 1456 validated image filenames.


In [60]:
train_generator.next()[0].shape,train_generator.next()[1].shape


((1, 300, 300, 3), (1, 4))

In [61]:
l=0
for batch in train_data_gen.flow_from_dataframe(
    train_data,
    train_dir,
    target_size=(image_size,image_size),
    x_col = 'image_id',
    y_col = ['healthy','multiple_diseases','rust','scab'],
    class_mode = 'raw',
    shuffle=False,
    subset='training',
    batch_size=batch_size,
    save_to_dir='/Users/pawelpawlus/Documents/asseco/plant_pathology_classification/preview',
    save_format='jpeg',
    save_prefix='train'):
    l+=1
    if l < 5:
        continue
    else:
        break





Found 1456 validated image filenames.


In [12]:
val_generator = train_data_gen.flow_from_dataframe(
    val_data,
    train_dir,
    target_size=image_size,
    x_col = 'image_id',
    y_col = ['healthy','multiple_diseases','rust','scab'],
    class_mode = 'raw',
    shuffle=False,
    batch_size=batch_size)

Found 365 validated image filenames.


In [13]:
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
LR_reduce=ReduceLROnPlateau(monitor='val_accuracy',
                            factor=.5,
                            patience=10,
                            min_lr=.000001,
                            verbose=1)

ES_monitor=EarlyStopping(monitor='val_loss',
                          patience=20)

reg = .0005

In [14]:
#x[0]

In [15]:
#plt.imshow(x[0])

In [16]:
# X = np.array(x)

In [17]:
# X.shape

In [18]:
# y = np.array(data_full.drop(['image_id'], axis=1))

In [19]:
# x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = 20, test_size=0.2)

In [20]:
# model = Sequential()

# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu", input_shape=(300,300,3), border_mode ='same'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu", border_mode='same'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=32, kernel_size=(5, 5), activation="relu",  border_mode ='same'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=8, kernel_size=(5, 5), activation="relu", border_mode='same'))

# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=8, kernel_size=(5, 5), activation="relu", border_mode='same'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(0.2))
# model.add(Dense(8, activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(0.2))

# model.add(Dense(4, activation='softmax'))

In [21]:
# from keras.regularizers  import l2
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(5,5),activation='relu', input_shape=(300, 300, 3), kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Conv2D(128, kernel_size=(5,5),activation='relu', kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
# model.add(Dropout(.25))

# model.add(Conv2D(32, kernel_size=(3,3),activation='relu', kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Conv2D(128, kernel_size=(3,3),activation='relu',kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
# model.add(Dropout(.25))


# model.add(Conv2D(128, kernel_size=(5,5),activation='relu', kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Conv2D(512, kernel_size=(5,5),activation='relu',kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
# model.add(Dropout(.25))

# model.add(Conv2D(128, kernel_size=(3,3),activation='relu',kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Conv2D(512, kernel_size=(3,3),activation='relu',kernel_regularizer=l2(reg)))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(MaxPooling2D(pool_size=(2,2), padding='SAME'))
# model.add(Dropout(.25))

# model.add(Flatten())
# model.add(Dense(300,activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(.25))
# model.add(Dense(200,activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(.25))
# model.add(Dense(100,activation='relu'))
# model.add(BatchNormalization(axis=-1,center=True,scale=False))
# model.add(Dropout(.25))
# model.add(Dense(4,activation='softmax'))


In [34]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(300,300,3),activation='relu'))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [35]:
# model = Sequential()

# model.add(GlobalAveragePooling2D(input_shape =(300,300,3)))

# #model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(8, activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(4, activation='softmax'))

In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 300, 300, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 298, 298, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 149, 149, 64)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1420864)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                22733840  
_________________________________________________________________
dropout_8 (Dropout)          (None, 16)               

In [37]:
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [38]:
# model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [39]:
results = model.fit_generator(train_generator,epochs=150,
                              steps_per_epoch=train_generator.n//batch_size,
                              validation_data=val_generator,
                             validation_steps=val_generator.n//batch_size,
                              verbose = 1,
                              callbacks=[LR_reduce,ES_monitor])

Epoch 1/150
15/91 [===>..........................] - ETA: 9:02 - loss: 2.0087 - accuracy: 0.3625

KeyboardInterrupt: 

In [ ]:
#history = model.fit(x_train,y_train, epochs=10,batch_size=16, validation_data=(x_test,y_test))